In [1]:
from collatex import *
collation = Collation()
collation.add_plain_witness( "A", "The quick brown fox jumped over the lazy dog.")
collation.add_plain_witness( "B", "The brown fox jumped over the dog." )
collation.add_plain_witness( "C", "The bad fox jumped over the lazy dog." )
table = collate(collation)
print(table)

+---+-----+-------+-------+---------------------+------+------+
| A | The | quick | brown | fox jumped over the | lazy | dog. |
| B | The | -     | brown | fox jumped over the | -    | dog. |
| C | The | bad   | -     | fox jumped over the | lazy | dog. |
+---+-----+-------+-------+---------------------+------+------+


In [2]:
collation = Collation()
A_content = "The quick brown fox jumped over the lazy dog."
B_content = "The brown fox jumped over the dog."
C_content = "The bad fox jumped over the lazy dog." 
collation.add_plain_witness( "A", A_content )
collation.add_plain_witness( "B", B_content )
collation.add_plain_witness( "C", C_content )
table = collate(collation)
print(table)

+---+-----+-------+-------+---------------------+------+------+
| A | The | quick | brown | fox jumped over the | lazy | dog. |
| B | The | -     | brown | fox jumped over the | -    | dog. |
| C | The | bad   | -     | fox jumped over the | lazy | dog. |
+---+-----+-------+-------+---------------------+------+------+


In [16]:
import re

def tokenize(input):
    return [create_token(token) for token in re.findall('\S+\s*',input)]

def create_token(input):
    return {"t": input}

collation = Collation()
A_content = "The quick brown fox jumped over the lazy dog."
B_content = "The brown fox jumped over the dog."
C_content = "The bad fox jumped over the lazy dog." 

witness_list = []
witness_list.append({"id": "A", "tokens": tokenize(A_content)})

json_input = {"witnesses": witness_list}
print(json_input)

{'witnesses': [{'id': 'A', 'tokens': [{'t': 'The '}, {'t': 'quick '}, {'t': 'brown '}, {'t': 'fox '}, {'t': 'jumped '}, {'t': 'over '}, {'t': 'the '}, {'t': 'lazy '}, {'t': 'dog.'}]}]}


In [17]:
import re

def normalize(input):
    return input.lower()

def tokenize(input):
    return [create_token(token) for token in re.findall('\S+\s*',input)]

def create_token(input):
    return {"t": input, "n": normalize(input)}

collation = Collation()
A_content = "The quick brown fox jumped over the lazy dog."
B_content = "The brown fox jumped over the dog."
C_content = "The bad fox jumped over the lazy dog." 

witness_list = []
witness_list.append({"id": "A", "tokens": tokenize(A_content)})
witness_list.append({"id": "B", "tokens": tokenize(B_content)})
witness_list.append({"id": "C", "tokens": tokenize(C_content)})

json_input = {"witnesses": witness_list}
# print(json_input)
table = collate(json_input)
print(table)

+---+-----+-------+-------+---------------------+------+------+
| A | The | quick | brown | fox jumped over the | lazy | dog. |
| B | The | -     | brown | fox jumped over the | -    | dog. |
| C | The | bad   | -     | fox jumped over the | lazy | dog. |
+---+-----+-------+-------+---------------------+------+------+


In [18]:
import re

def normalize(input):
    return input.lower()

def tokenize(input):
    return [create_token(token) for token in re.findall('\S+\s*',input)]

def create_token(input):
    return {"t": input, "n": normalize(input)}

collation = Collation()
A_content = "Look, a gray koala!"
B_content = "Look, a big grey koala!"
C_content = "Look, a big wombat!" 

witness_list = []
witness_list.append({"id": "A", "tokens": tokenize(A_content)})
witness_list.append({"id": "B", "tokens": tokenize(B_content)})
witness_list.append({"id": "C", "tokens": tokenize(C_content)})

json_input = {"witnesses": witness_list}
# print(json_input)
table = collate(json_input)
print(table)

+---+---------+------+---------+--------+
| A | Look, a | gray | -       | koala! |
| B | Look, a | big  | grey    | koala! |
| C | Look, a | big  | wombat! | -      |
+---+---------+------+---------+--------+


In [21]:
import re
import string

def normalize(input):
    input = re.sub('[' + string.punctuation + '']','',input)
    animals = ['koala', 'wombat']
    if input in animals:
        return 'ANIMAL'
    else:
        return input.lower()

def tokenize(input):
    return [create_token(token) for token in re.findall('\S+\s*',input)]

def create_token(input):
    return {"t": input, "n": normalize(input)}

collation = Collation()
A_content = "Look, a gray koala!"
B_content = "Look, a big grey koala!"
C_content = "Look, a big wombat!" 

witness_list = []
witness_list.append({"id": "A", "tokens": tokenize(A_content)})
witness_list.append({"id": "B", "tokens": tokenize(B_content)})
witness_list.append({"id": "C", "tokens": tokenize(C_content)})

json_input = {"witnesses": witness_list}
# print(json_input)
table = collate(json_input)
print(table)

error: multiple repeat at position 10